### 4-Gewinnt Widget-Klasse
`Widget(game)` erstellt ein Widget, welches das Spielbrett graphisch darstellt:
- Klicken mit der Maus in eine Spalte platzert dort einen Stein.
- Dr&uuml;cken von 4,5,..., oder 9 startet ein neues Spiel mit der entsprechenden Anzahl Spalten.
- Dr&uuml;cken von cntr-4,cntr-5,..., oder cntr-9  startet ein neues Spiel, wo Spieler 1 beginnt.

`Widget(game)` setzt `game.callback = self.update(event, data)` und ruft `game.new_game` auf.  
`update(event, data)` ruft dann die Methode mit Namen `event` mit den Keyword-Argumenten `**data` auf.  
Die *Magic*-Methode `_ipython_display_` definiert, wie das Widget dargestellt wird.

In [ ]:
from viergewinnt import Game
help(Game)

In [ ]:
from ipycanvas import Canvas
from ipywidgets import Output
from viergewinnt import Game

# Output-Widgets
layout = {'border': '1px solid black'}
out = Output(layout=layout)

class Widget:
    def __init__(self, game, color_0='yellow', color_1='red', size=300):
        self.player_color = {0: color_0, 1: color_1}
       
        self.canvas = Canvas(width=size, height=size, layout=layout)
        self.canvas.on_key_down(self.on_key_down)
        self.canvas.on_mouse_down(self.on_mouse_down)
        
        self.game = game
        self.game.callback = self.update
        self.game.new_game()
       
    def draw_board(self, rows, cols):
        fg = self.feldgroesse
        canvas = self.canvas
        canvas.text_baseline = 'alphabetic'
        canvas.font = font = '{}px serif'.format(fg)
        canvas.clear()
        canvas.fill_style = 'blue'
        canvas.fill_rect(0, 0, canvas.width, canvas.height)

        # Zeichne senkrechte Linien
        for i in range(1, cols):
            x = i * fg
            canvas.stroke_lines([(x, 0), (x, canvas.height)])

        # Zeichne waagerechte Linien
        for i in range(1, rows+1):
            y = i * fg
            canvas.stroke_lines([(0, y), (canvas.width, y)])

        # Spaltenbeschriftung
        zahlen = list(range(1, cols + 1))  # Speichert die Zahlen von 1 bis 7
        canvas.text_baseline = 'alphabetic'
        canvas.font = font = '{}px serif'.format(fg)
        for i, zahl in enumerate(zahlen):
            canvas.fill_style = 'black'
            canvas.fill_text(zahl, fg/4 + i*fg, 4/5*fg)

        # Aussenrahmen
        canvas.stroke_style = 'black'
        canvas.stroke_rect(0, 0, canvas.width, canvas.height)
        
        # weisse Steine zeichnen (leere Felder)
        for row in range(rows):
            for col in range(cols):
                self.draw_piece((row, col))
                
    def draw_piece(self, pos, color='white'):
        canvas = self.canvas
        row, col = pos
        canvas.fill_style = color
        
        x = (col + 0.5) * self.feldgroesse 
        y = canvas.height -  (row + 0.5) * self.feldgroesse
        canvas.fill_circle(x, y, self.radius)
    
    # vom Callback benutzte Methoden
    def new_game(self, player, rows, cols):
        self.feldgroesse = self.canvas.width / cols
        self.radius = self.feldgroesse / 3.5
        self.draw_board(rows, cols)
        out.clear_output()
    
    def place_piece(self, player, pos):
        color = self.player_color[player]
        self.draw_piece(pos, color)
        
    def update_result(self, result):
        color = self.player_color.get(result)
        msg = 'Tie' if color is None else '{} wins'.format(color)
        print(msg)
        
    # Maus und Tastatursteuerung    
    @out.capture()
    def on_key_down(self, key, *flags):
        if key in '456789':
            self.game.new_game(cols=int(key), player=int(flags[1]))
        
    @out.capture()    
    def on_mouse_down(self, x, y):
        col = int(x // self.feldgroesse)
        self.game.drop_piece(col)
     
    # Callback, delegiert an `new_game`, `place_piece` und 'update_result`
    @out.capture()
    def update(self, event, data):
        # print('event: {}, data: {}'.format(event, data))
        getattr(self, event)(**data)
        
    def _ipython_display_(self):
        display(self.canvas, out)
        self.canvas.focus()

In [ ]:
game= Game()
Widget(game)